In [2]:
import os

#from IPython.display import set_matplotlib_formats
import jax
import jax.numpy as jnp
from jax import random, vmap
from jax.scipy.special import logsumexp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math
import scipy

import numpyro
from numpyro.diagnostics import hpdi
import numpyro.distributions as dist
from numpyro import handlers
from numpyro.infer import MCMC, NUTS
from sklearn.model_selection import train_test_split
numpyro.set_platform("cpu")

/Users/heningwang/Documents/GitHub/numpyro_adjective_modelling/jax-metal/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
# Minimal experimentation with MCMC inference with categorical data

data = jnp.array([0,1,2,2,2,3,4])

def softmax(array):
    return jnp.exp(array) / jnp.sum(jnp.exp(array))

def model(data):
    mu1 = numpyro.sample('mu1', dist.Uniform(0, 1))
    mu2 = numpyro.sample('mu2', dist.Uniform(0, 1))
    mu3 = numpyro.sample('mu3', dist.Uniform(0, 1))
    mu4 = numpyro.sample('mu4', dist.Uniform(0, 1))
    # softmax the mus
    probs = softmax(jnp.array([mu1, mu2, mu3, mu4]))

    with numpyro.plate('data', len(data)):
        samples = numpyro.sample('samples', dist.Categorical(probs), obs=data)

kernel = NUTS(model)
mcmc = MCMC(kernel, num_warmup=10, num_samples=10)

mcmc.run(random.PRNGKey(0), data)
mcmc.print_summary()
samples = mcmc.get_samples()


/var/folders/jc/6ctllcx143xbrgx01x69skg80000gn/T/ipykernel_37847/1111702470.py:21: UserWarning: Site samples: Out-of-support values provided to log prob method. The value argument should be within the support.
  mcmc.run(random.PRNGKey(0), data)


RuntimeError: Cannot find valid initial parameters. Please check your model again.

In [24]:
# Minimal experimentation with MCMC inference with categorical data

data = jnp.array([0,1,2,2,2,3,4])

def softmax(array):
    return jnp.exp(array) / jnp.sum(jnp.exp(array))

import jax.numpy as jnp
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS, Predictive

def model(data):
    # Define latent continuous parameters, e.g., alpha, beta
    alpha = numpyro.sample('alpha', dist.Normal(0, 1))
    beta = numpyro.sample('beta', dist.Normal(0, 1))
    gamma = numpyro.sample('gamma', dist.Normal(0, 1))
    delta = numpyro.sample('delta', dist.Normal(0, 1))
    
    # Combine the parameters into a vector
    logits = jnp.array([alpha, beta, gamma, delta])
    
    # Compute probs using a softmax function to get a probability vector over four categories
    probs = jax.nn.softmax(logits)
    
    with numpyro.plate('data', len(data)):
        # This is the likelihood function, sample from a categorical distribution with probs as parameter, conditioned on observed data
        samples = numpyro.sample('samples', dist.Categorical(probs=probs), obs=data)
        
kernel = NUTS(model)
mcmc = MCMC(kernel, num_warmup=10, num_samples=10)

mcmc.run(random.PRNGKey(0), data)
mcmc.print_summary()
samples = mcmc.get_samples()


/var/folders/jc/6ctllcx143xbrgx01x69skg80000gn/T/ipykernel_37847/2915075981.py:32: UserWarning: Site samples: Out-of-support values provided to log prob method. The value argument should be within the support.
  mcmc.run(random.PRNGKey(0), data)


RuntimeError: Cannot find valid initial parameters. Please check your model again.